# Importing Libraries

In [18]:
pip install tensorflow


     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 0.4/377.1 MB 7.8 MB/s eta 0:00:49
   ---------------------------------------- 1.3/377.1 MB 17.1 MB/s eta 0:00:23
   ---------------------------------------- 2.5/377.1 MB 19.9 MB/s eta 0:00:19
   ---------------------------------------- 4.6/377.1 MB 26.8 MB/s eta 0:00:14
    --------------------------------------- 5.9/377.1 MB 27.0 MB/s eta 0:00:14
    --------------------------------------- 7.6/377.1 MB 28.6 MB/s eta 0:00:13
    --------------------------------------- 8.8/377.1 MB 28.3 MB/s eta 0:00:14
   - -------------------------------------- 9.9/377.1 MB 27.6 MB/s eta 0:00:14
   - -------------------------------------- 10.4/377.1 MB 27.3 MB/s eta 0:00:14
   - -------------------------------------- 10.8/377.1 MB 25.1 MB/

In [20]:
import numpy as np
import os
import datetime
import random as rn
import imageio
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

import warnings
warnings.filterwarnings("ignore")

In [22]:
np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)

In [24]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32  # Experiment with the batch size


In [25]:
def generator(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0, 30)]  # Use all 30 images per video

    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size  # Calculate the number of batches

        for batch in range(num_batches):  # Iterate over the number of batches
            batch_data = np.zeros((batch_size, len(img_idx), 120, 160, 3))  # Adjust this to desired input shape
            batch_labels = np.zeros((batch_size, 5))  # One-hot encoding of labels

            for folder in range(batch_size):  # Iterate over the batch_size
                imgs = sorted(os.listdir(os.path.join(source_path, t[folder + (batch * batch_size)].split(';')[0])))
                for idx, item in enumerate(img_idx):  # Iterate over the frames/images of a folder
                    image = imageio.imread(os.path.join(source_path, t[folder + (batch * batch_size)].split(';')[0], imgs[item])).astype(np.float32)
                    image_resized = resize(image, (120, 160))  # Resize images to 120x160

                    batch_data[folder, idx, :, :, :] = image_resized / 255.0  # Normalize and add image

                label = int(t[folder + (batch * batch_size)].split(';')[2])
                batch_labels[folder, label] = 1  # One-hot encoding

            yield batch_data, batch_labels  # Yield the batch data and labels

        # Handle remaining data
        if len(t) % batch_size != 0:
            remaining_data = len(t) % batch_size
            batch_data = np.zeros((remaining_data, len(img_idx), 120, 160, 3))
            batch_labels = np.zeros((remaining_data, 5))

            for folder in range(remaining_data):
                imgs = sorted(os.listdir(os.path.join(source_path, t[folder + (num_batches * batch_size)].split(';')[0])))
                for idx, item in enumerate(img_idx):
                    image = imageio.imread(os.path.join(source_path, t[folder + (num_batches * batch_size)].split(';')[0], imgs[item])).astype(np.float32)
                    image_resized = resize(image, (120, 160))

                    batch_data[folder, idx, :, :, :] = image_resized / 255.0

                label = int(t[folder + (num_batches * batch_size)].split(';')[2])
                batch_labels[folder, label] = 1

            yield batch_data, batch_labels


In [32]:
def create_conv3d_model():
    model = Sequential()
    model.add(Conv3D(32, (3, 3, 3), padding='same', input_shape=(30, 120, 160, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    model.add(Conv3D(64, (3, 3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    model.add(Conv3D(128, (3, 3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(5, activation='softmax'))  # 5 classes

    return model


In [33]:
model = create_conv3d_model()
optimiser = optimizers.Adam(learning_rate=0.001)  # Use 'learning_rate' instead of 'lr'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_9 (Conv3D)               │ (None, 30, 120, 160,   │         2,624 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 30, 120, 160,   │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 30, 120, 160,   │             0 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_9 (MaxPooling3D)  │ (None, 15, 60, 80, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_10 (Conv3D)              │ (None, 15, 60, 80, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 15, 60, 80, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 15, 60, 80, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_10 (MaxPooling3D) │ (None, 8, 30, 40, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_11 (Conv3D)              │ (None, 8, 30, 40, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 8, 30, 40, 128) │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 8, 30, 40, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_11 (MaxPooling3D) │ (None, 4, 15, 20, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 153600)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │    78,643,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,926,469 (301.08 MB)

 Trainable params: 78,926,021 (301.08 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [34]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20  # Choose the number of epochs
print('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [31]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1, mode='auto', min_lr=0.00001)
callbacks_list = [checkpoint, LR]

steps_per_epoch = num_train_sequences // batch_size
validation_steps = num_val_sequences // batch_size

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=model_init_2024-05-2819_31_36.519366/model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5

In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
          callbacks=callbacks_list, validation_data=val_generator, validation_steps=validation_steps, 
          class_weight=None, workers=1, initial_epoch=0)